查看数据集的维度/内容

In [3]:
import numpy as np
data_name = 'NYCBike1' #'NYCBike1' 'NYCBike2' 'NYCTaxi' 'METRLA' 'PEMSBAY'

1. 加载train/val/test.npz

In [4]:
data_train = np.load(f'./{data_name}/train.npz')
for file in data_train.files:
    print(file, data_train[file].shape, data_train[file].dtype)

x (3023, 19, 128, 4) float64
y (3023, 1, 128, 4) float64
x_offsets (19, 1) int64
y_offsets (1, 1) int64


2. 加载邻接矩阵

In [38]:
data_adj = np.load(f'./{data_name}/adj_mx.npz')
for file in data_adj.files:
    print(file, data_adj[file].shape, data_adj[file].dtype)
# 对于NYCBike1(16*8), graph的node顺序按照h16w8的gird顺序
# np.savetxt(f'graph/graph_{data_name}.txt', data_adj['adj_mx'], fmt='%0.2f')

adj_mx (325, 325) float64


2.1 计算edges数目

In [39]:
def count_edges(adj_matrix, direted):
    assert len(adj_matrix.shape) == 2 and adj_matrix.shape[0] == adj_matrix.shape[1], "Matrix must be square"
    np.fill_diagonal(adj_matrix, 0)
    edge_matrix = (adj_matrix != 0).astype(int)
    edge_count = np.sum(edge_matrix) // (1 if direted else 2)
    return edge_count

direted = True if data_name in ['METRLA', 'PEMSBAY'] else False
print("Number of edges:", count_edges(data_adj['adj_mx'], direted))

Number of edges: 2369


2.2 floyd_warshall算法测试

In [40]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 
import numpy as np
from scipy.sparse.csgraph import floyd_warshall
def get_shortpath_num(graph, dataset):
    if dataset in ['METRLA', 'PEMSBAY']:
        graph = modify_graph_numpy(graph)
    shortest_path = floyd_warshall(graph, return_predecessors=False)
    return shortest_path

# for METRLA/PEMSBAY dataset
# metr graph原形式：见DCRNN
def modify_graph_numpy(graph):
    adj_matrix = np.array(graph, dtype=float)
    
    non_zero_mask = adj_matrix != 0
    adj_matrix[non_zero_mask] = np.sqrt(np.abs(np.log(adj_matrix[non_zero_mask])))

    # 将原始矩阵中为0的元素设置为无穷大
    adj_matrix[~non_zero_mask] = np.inf
    # 将对角线上的元素设置为0
    np.fill_diagonal(adj_matrix, 0)
    return adj_matrix

dist_matrix = get_shortpath_num(data_adj['adj_mx'], data_name)
np.savetxt(f'graph/floyd_{data_name}.txt', dist_matrix, fmt='%0.2f')